<a href="https://colab.research.google.com/github/zainali60/Flower-Recognition/blob/main/Flowers_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Assignment: Flowers Recognition <br>
Dataset Description:<br>

This dataset contains 4242 images of flowers.<br>
The data collection is based on the data flicr, google images, yandex images.<br>
You can use this datastet to recognize plants from the photo.<br>

Attribute Information:<br>
The pictures are divided into five classes: chamomile, tulip, rose, sunflower, dandelion.<br>
For each class there are about 800 photos. Photos are not high resolution, about 320x240 pixels. <br>
<b>Also explore how to resize images in tensorflow and then resize all the images to a same size. </b> <br>
This is a Multiclass Classification Problem.<br>




WORKFLOW : <br>
Load Data <br>
Split into 60 and 40 ratio.<br>
Encode labels.<br>
Create Model<br>
Compilation Step (Note : Its a Multiclass Classification problem , select loss , metrics according to it)<br>
Train the Model.<br>
If the model gets overfit tune your model by changing the units , No. of layers , epochs , add dropout layer or add Regularizer according to the need .<br>
Prediction should be > 85%<br>
Evaluation Step<br>
Prediction<br>




Data : <br>
https://drive.google.com/file/d/1-OX6wn5gA-bJpjPNfSyaYQLz-A-AB_uj/view?usp=sharing

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from os import listdir
import os
from os.path import join
import cv2
import pandas as pd
import pathlib
from pathlib import Path

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# Loading Data And Defining Features And Labels
directory = Path("/content/drive/MyDrive/flowers")

image_names = []
data_images = []
labels = []

# Iterating Over Directory To Extract Sub Directories
for dir in directory.iterdir():
  image_names.append(dir.name)
  print(dir.name)
# Iterating Over Sub Directories To Extract Lables
  for imgpath in dir.iterdir():
    if imgpath.name.endswith("jpg"):
      labels.append(dir.name)
      imgarr = cv2.imread(str(imgpath), cv2.IMREAD_GRAYSCALE)
      imgarr = cv2.resize(imgarr, (150,150))
      data_images.append(imgarr)

In [ ]:
df = pd.DataFrame(labels)
label = df[0].map({'daisy':0,'rose':1,'sunflower':2,'tulip':3,'dandelion':4})

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(data_images,label,test_size=.40,random_state=1)

In [ ]:
X_train = np.asarray(X_train).astype('float32')/255
y_train = np.asarray(y_train)
X_test = np.asarray(X_test).astype('float32')/255
y_test = np.asarray(y_test)

In [ ]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
X_train = X_train.reshape(2593,150*150)
X_test = X_test.reshape(1730,150*150)

In [ ]:
X_train[0]

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import losses,optimizers,regularizers
network =Sequential()
network.add(Dense(64,activation="relu",input_shape=(X_train.shape[1],)))
#network.add(Dropout(0.2))
network.add(Dense(32,kernel_regularizer=regularizers.l2(0.001),activation='relu'))
#network.add(Dense(32,kernel_regularizer=regularizers.l2(0.001),activation='relu'))
#network.add(Dropout(0.2))
#network.add(Dense(16,activation='relu'))
#network.add(Dropout(0.2))
network.add(Dense(5,activation="softmax"))

In [ ]:
#network.summary()

In [ ]:
network.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [ ]:
batch_size = 20
epochs = 30
history=network.fit(X_train, y_train,batch_size=batch_size,epochs=epochs)

In [ ]:
history_dict = history.history
history_dict.keys()
[u'accuracy', u'loss']

In [ ]:
import matplotlib.pyplot as plt
history_dict = history.history
loss_values = history_dict['loss']
val_accuracy = history_dict['accuracy']
epochs = range(1, len(history_dict['accuracy']) + 1)
plt.plot(epochs, loss_values, 'r', label='loss')
plt.plot(epochs, val_accuracy, 'b', label='accuracy')
plt.title('loss and accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid()
plt.show()

In [ ]:
network.evaluate(X_test,y_test)

In [ ]:
pred = network.predict_classes(X_test[:10])
for i in range(len(pred)):
    print(pred[i],'==>',y_test[i])

In [ ]:
plt.figure(figsize=(20,5))
plt.subplot(1, 2, 1 )
plt.hist(y_test[:10])
plt.xlabel('original target value')
plt.ylabel('count')
plt.subplot(1, 2, 2)
plt.hist(pred)
plt.xlabel('aggregated target value')
plt.ylabel('count')
plt.show()